In [1]:
from flask import Flask, render_template, request, jsonify
from flaskext.mysql import MySQL
from surprise import dump, KNNBasic
from tqdm import tqdm
from operator import itemgetter

import pickle
import pymysql  # mysql error handling
import functools
import itertools
import json
import random

import numpy as np
import pandas as pd

In [2]:
from rec_utils import *
from db_helper import DbHelper

In [3]:
app = Flask(__name__)
app.config.from_pyfile('config.py')

mysql = MySQL()
mysql.init_app(app)

conn = mysql.connect()
cursor = conn.cursor()

In [4]:
db = DbHelper(conn, cursor)


## Registro

In [5]:
socio = {"apPaterno": "Mendoza", "apMaterno": "Hernandez", "nombre": "Alma", 
        "edad": 22, "genero": "F", "email": "almis@gmail.com", "passwd": "frio"}
db.register(socio)

{'email_used': False, 'success': True}

In [6]:
db.register(socio)

(1062, "Duplicate entry 'almis@gmail.com' for key 'socio.c_uniq_email_passwd'")


{'email_used': True, 'success': False}

In [7]:
socio = {"apPaterno": "Fragoso", "apMaterno": "Hernandez", "nombre": "Abigail", 
        "edad": 19, "genero": "F", "email": "almis@gmail.com", "passwd": "frio"}
db.register(socio)

(1062, "Duplicate entry 'almis@gmail.com' for key 'socio.c_uniq_email_passwd'")


{'email_used': True, 'success': False}

In [8]:
db.delete('delete from socio where passwd = "frio"')

## Login

In [9]:
email, passwd = 'almis@gmail.com', 'frio'
db.login(email, passwd)

{'email_found': False, 'correct_passwd': False, 'idSocio': False}

In [10]:
passwd = 'incorrecto'
db.login(email, passwd)

{'email_found': False, 'correct_passwd': False, 'idSocio': False}

In [11]:
email = 'noexiste@gmail.com'
db.login(email, 'falsooo')

{'email_found': False, 'correct_passwd': False, 'idSocio': False}

## product info

In [12]:
raw_iid = 'B005FIWTHO'
db.get_product_info(raw_iid)

{'idProducto': 'B005FIWTHO',
 'nombre': 'Quicken Premier 2012',
 'marca': 'Intuit',
 'precioUnitario': 44.96298689479438,
 'idSubCat': 17}

In [13]:
raw_iids = ['B005FIWTHO','B0053WX3AY','B00EZPXYP4','B00BFNCFZ4']
db.get_products_info(raw_iids)

[{'idProducto': 'B0053WX3AY',
  'nombre': 'Dragon NaturallySpeaking Home, European Version 11.5',
  'marca': 'Nuance Communications, Inc.',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17},
 {'idProducto': 'B005FIWTHO',
  'nombre': 'Quicken Premier 2012',
  'marca': 'Intuit',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17},
 {'idProducto': 'B00BFNCFZ4',
  'nombre': 'VideoStudio Pro X6 [OLD VERSION]',
  'marca': 'Corel',
  'precioUnitario': 19.95,
  'idSubCat': 17},
 {'idProducto': 'B00EZPXYP4',
  'nombre': 'NORTON 360 21.0 EN 1U 3LIC MM',
  'marca': 'Symantec',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17}]

## Recomendaciones 

In [14]:
# load the algo
# file_name = 'algo.model'
# wdir = './model/'
# _, algo = dump.load(wdir + file_name)

# config to keep same experiments
my_seed = 0
random.seed(my_seed)
np.random.seed(my_seed)

algo, sims, trainset, testset = get_rec_sys_resources()

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [15]:
raw_uid = testset[0][0]
raw_iids = get_top_item_based(algo, raw_uid, trainset, sims)  # if raw_id not in trainset it raises error
raw_uid, raw_iids

('A2JSNRJV7DL5JZ',
 ['B005FIWTHO',
  'B0053WX3AY',
  'B00EZPXYP4',
  'B00BFNCFZ4',
  'B0013QQWQG',
  'B0123C60EW',
  'B015724RQI',
  'B000HCZ8EO',
  'B00LC9UU6C',
  'B000X86ZAS'])

If there's no uid in trainset ValueError is raised.

In [16]:
not_in_trainset = []
for uid in tqdm(map(lambda x: x[0], testset), total=len(testset)):
    try:
        get_top_item_based(algo, uid, trainset, sims)
    except ValueError as e:
        not_in_trainset.append(uid)


100%|██████████| 4378/4378 [01:10<00:00, 62.05it/s] 


In [17]:
len(not_in_trainset)

11

In [20]:
df_reviews = pd.read_csv('./model/software_reviews.csv')
uid_ratings = df_reviews[df_reviews.reviewerID == uid].asin.values
db.get_products_info(uid_ratings)[:5]

[{'idProducto': 'B000HCZ8EO',
  'nombre': 'Microsoft Office Home and Student 2007 [Old Version]',
  'marca': 'Microsoft',
  'precioUnitario': 149.99,
  'idSubCat': 17},
 {'idProducto': 'B004E9SKDM',
  'nombre': 'Microsoft Windows 7 Anytime Upgrade (Starter to Home Premium) [Online Code]',
  'marca': 'Microsoft',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17},
 {'idProducto': 'B008RA5A00',
  'nombre': 'Quicken Deluxe 2013',
  'marca': 'Intuit',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17},
 {'idProducto': 'B00BR082FW',
  'nombre': 'CCleaner Professional System Optimization Tool  Unlimited Home Use',
  'marca': 'Piriform',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17}]

In [21]:
iid_recs = get_top_item_based(algo, uid, trainset, sims, k=5)  # if raw_id not in trainset it raises error
db.get_products_info(iid_recs)

[{'idProducto': 'B002SRNS7A',
  'nombre': 'Norton Ghost 15.0 - 1 PC',
  'marca': 'Symantec',
  'precioUnitario': 179.95,
  'idSubCat': 17},
 {'idProducto': 'B005FIWTHO',
  'nombre': 'Quicken Premier 2012',
  'marca': 'Intuit',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17},
 {'idProducto': 'B00M76N6MO',
  'nombre': 'Quicken For Mac Personal Finance &amp; Budgeting Software 2015 [Old Version]',
  'marca': 'Intuit',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17},
 {'idProducto': 'B00MUY6LL6',
  'nombre': 'Trend Micro Premium Security 2015 - 5 Devices [OLD VERSION][OLD VERSION]',
  'marca': 'Trend Micro',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17},
 {'idProducto': 'B00U7LCE6A',
  'nombre': 'CCleaner Free [Download]',
  'marca': 'Piriform',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17}]

## Agregar productos comprados

In [22]:
req = {"idSocio": "1", "idProductos": [], "cantidades": []}
cursor.execute("SELECT idProducto FROM producto LIMIT 5")
iids = list(map(itemgetter(0), cursor.fetchall()))
req["idProductos"] = iids
req["cantidades"] = [1 for _ in iids]
req

{'idSocio': '1',
 'idProductos': ['1', '2', '3', '4', '5'],
 'cantidades': [1, 1, 1, 1, 1]}

In [23]:
db.insert_hist(req)

{'success': True}

In [24]:
db.select('select * from historial')

(('1', '1', datetime.datetime(2021, 1, 8, 17, 9, 56), 1),
 ('1', '2', datetime.datetime(2021, 1, 8, 17, 9, 56), 1),
 ('1', '3', datetime.datetime(2021, 1, 8, 17, 9, 56), 1),
 ('1', '4', datetime.datetime(2021, 1, 8, 17, 9, 56), 1),
 ('1', '5', datetime.datetime(2021, 1, 8, 17, 9, 56), 1))

In [25]:
db.delete('delete from historial')

## Pendiente

In [26]:
res = {"idSocio": "1", "idProducto": "1"}
db.insert_pendiente(res)

{'success': True}

In [27]:
db.select('select * from pendiente')

(('1', '1'),)

In [28]:
db.delete('delete from pendiente')